# Dynamic functional connectivity analysis

#### fMRI Data University of Leuven

We want to classify three different states of dynamic functional connectivity: hypoconnectivity, intermediate and hyperconnectivity. Then we want to compare between-groups to see if there are significant differences in the states, i.e. if the ASD group spent more time in a hypo, intermediate or hyperconnected state in comparison with the TD group. 

Finally, we will obtain different indices like the Mean Dwell Time (MDT) and the Probability of transition (PT) from one state to another and again compare between-groups for significant differences.  

In [55]:
import numpy as np
import pandas as pd

# These lines do some fancy plotting.
import matplotlib
%matplotlib inline
import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight')

import plotly.graph_objects as go

#Import stats from scipy library
from scipy import stats

#Read .csv file 
project = pd.read_csv('rvaTPJ_no_rvpTPJ_374_001_008.csv')

#Change column names for simplicity. Delete the first part of the names 'R2R target ROI mask_rvaTPJ : + mask_rvpTPJ @ rest x'
project.columns = [col.replace('R2R target ROI mask_rvaTPJ_no : + mask_rvpTPJ_no @ rest x', '') for col in project.columns]

#load first five rows of the data set
project.head()

,SRS_RAW_TOTAL,SRS_AWARENESS,SRS_COGNITION,SRS_COMMUNICATION,SRS_MOTIVATION,SRS_MANNERISMS,AQ_TOTAL,rest,Time1,Time2,...,Time215,Time216,Time217,Time218,Time219,Time220,Time221,Time222,Time223,Time224
0,68,10,15,22,15,6,33,-0.372887,0.898793,0.928372,...,-0.696586,-0.655891,-0.593027,-0.507097,-0.394806,-0.249987,-0.064806,0.167787,0.451302,0.779237
1,55,4,11,17,14,9,32,-0.252600,-0.794247,-0.678755,...,-0.727096,-0.719751,-0.714117,-0.711336,-0.712312,-0.717918,-0.728663,-0.744058,-0.762272,-0.780523
2,31,5,7,8,6,5,8,0.318661,-0.543222,-0.598560,...,1.074960,1.065311,1.072327,1.081409,1.080143,1.063993,1.035652,1.000526,0.961694,0.916893
3,72,9,15,26,10,12,30,0.344258,-0.016524,-0.025186,...,0.976433,1.009060,1.056994,1.112173,1.161631,1.190570,1.188566,1.153639,1.090470,1.006295
4,88,12,20,30,16,10,31,0.683666,0.743461,0.733636,...,1.652535,1.585614,1.517732,1.449486,1.378855,1.300475,1.205809,1.084392,0.927029,0.736244


In [56]:
#Create a new data frame 'windows' with the last 211 columns of the 'project' data frame 
#i.e. only the z-scores per subject, per window (211 windows in total). 

windows = project[project.columns[-224:project.columns.size]]

#Create two new columns with the average and standard deviation across windows. 
#i.e dynamicFC strength and dynamicFC variability

project['dynStr'] = np.mean(windows, axis = 1)
project['dynSD'] = np.std(windows, axis = 1)
project.head()

,SRS_RAW_TOTAL,SRS_AWARENESS,SRS_COGNITION,SRS_COMMUNICATION,SRS_MOTIVATION,SRS_MANNERISMS,AQ_TOTAL,rest,Time1,Time2,...,Time217,Time218,Time219,Time220,Time221,Time222,Time223,Time224,dynStr,dynSD
0,68,10,15,22,15,6,33,-0.372887,0.898793,0.928372,...,-0.593027,-0.507097,-0.394806,-0.249987,-0.064806,0.167787,0.451302,0.779237,-0.341972,0.674439
1,55,4,11,17,14,9,32,-0.252600,-0.794247,-0.678755,...,-0.714117,-0.711336,-0.712312,-0.717918,-0.728663,-0.744058,-0.762272,-0.780523,-0.377624,0.637034
2,31,5,7,8,6,5,8,0.318661,-0.543222,-0.598560,...,1.072327,1.081409,1.080143,1.063993,1.035652,1.000526,0.961694,0.916893,0.354312,0.689594
3,72,9,15,26,10,12,30,0.344258,-0.016524,-0.025186,...,1.056994,1.112173,1.161631,1.190570,1.188566,1.153639,1.090470,1.006295,0.296787,0.577184
4,88,12,20,30,16,10,31,0.683666,0.743461,0.733636,...,1.517732,1.449486,1.378855,1.300475,1.205809,1.084392,0.927029,0.736244,0.854236,0.763824


In [57]:
#Select windows with low, medium or high values. 
#low = windows<=-0.401179
#medium = (windows >= -0.401189) & (windows <= 0.301016)  
#high = windows>=0.301026 

#low = windows<=-0.227111
#medium = (windows >= -0.227121) & (windows <= 0.229148)  
#high = windows>=0.229158 

high_negative = windows<-0.5
moderate_negative = (windows >= -0.5) & (windows < -0.25)  
low_uncorrelated= (windows >= -0.25) & (windows <= 0.25) 
moderate_positive = (windows > 0.25) & (windows <= 0.5)
high_positive = windows>0.5


#Create three new columns with the counts of lows, mediums and highs windows respectively, per subject. 
prop_high_negative = high_negative.sum(axis = 1) / 224
prop_moderate_negative = moderate_negative.sum(axis = 1) / 224
prop_low_uncorrelated = low_uncorrelated.sum(axis = 1) / 224
prop_moderate_positive = moderate_positive.sum(axis = 1) / 224
prop_high_positive = high_positive.sum(axis = 1) / 224

project['prop_high_negative'] = prop_high_negative
project['prop_moderate_negative'] = prop_moderate_negative
project['prop_low_uncorrelated'] = prop_low_uncorrelated
project['prop_moderate_positive'] = prop_moderate_positive
project['prop_high_positive'] = prop_high_positive

#Check the proportions of wach connectivity state for each subject. 
#last5 = project[project.columns[-5:project.columns.size]]
#project['sum'] = np.sum(last5, axis =1)

project.head()

,SRS_RAW_TOTAL,SRS_AWARENESS,SRS_COGNITION,SRS_COMMUNICATION,SRS_MOTIVATION,SRS_MANNERISMS,AQ_TOTAL,rest,Time1,Time2,...,Time222,Time223,Time224,dynStr,dynSD,prop_high_negative,prop_moderate_negative,prop_low_uncorrelated,prop_moderate_positive,prop_high_positive
0,68,10,15,22,15,6,33,-0.372887,0.898793,0.928372,...,0.167787,0.451302,0.779237,-0.341972,0.674439,0.433036,0.129464,0.232143,0.084821,0.120536
1,55,4,11,17,14,9,32,-0.252600,-0.794247,-0.678755,...,-0.744058,-0.762272,-0.780523,-0.377624,0.637034,0.406250,0.218750,0.196429,0.066964,0.111607
2,31,5,7,8,6,5,8,0.318661,-0.543222,-0.598560,...,1.000526,0.961694,0.916893,0.354312,0.689594,0.129464,0.071429,0.191964,0.129464,0.477679
3,72,9,15,26,10,12,30,0.344258,-0.016524,-0.025186,...,1.153639,1.090470,1.006295,0.296787,0.577184,0.111607,0.066964,0.294643,0.187500,0.339286
4,88,12,20,30,16,10,31,0.683666,0.743461,0.733636,...,1.084392,0.927029,0.736244,0.854236,0.763824,0.053571,0.049107,0.129464,0.066964,0.700893


In [58]:
#Create new data frames for each group, autism spectrum disorder (asd) and typically developing (td).

#New data frames
asd = project.loc[:13]
td = project.loc[14:]

#First, we use Shapiro-Wilk test to test for normality. We want the p-value to be higher that 0.05 becasue that means we cannot reject
#the null hypothesis that the data comes from a normal distribution. Hence, the data comes from a normal distribution.  
shapiro_test = stats.shapiro(project['prop_high_negative']), stats.shapiro(project['prop_moderate_negative']), stats.shapiro(project['prop_low_uncorrelated']), stats.shapiro(project['prop_moderate_positive']), stats.shapiro(project['prop_high_positive'])
print('Normality test (Shapiro-Wilk):', shapiro_test)
print() 

# Then, we test for equality of variance with Levene's test. Again, if p-values > 0.05 means that the samples come from 
#populations with equal variances. 
levene_test = stats.levene(td['prop_high_negative'], asd['prop_high_negative']),  stats.levene(td['prop_moderate_negative'], asd['prop_moderate_negative']), stats.levene(td['prop_low_uncorrelated'], asd['prop_low_uncorrelated']), stats.levene(td['prop_moderate_positive'], asd['prop_moderate_positive']), stats.levene(td['prop_high_positive'], asd['prop_high_positive'])
print('Equality of variance test (Levene):',levene_test)

Normality test (Shapiro-Wilk): ((0.9619383215904236, 0.36658474802970886), (0.9520197510719299, 0.20646537840366364), (0.9730582237243652, 0.6450692415237427), (0.9342964291572571, 0.071134552359581), (0.870661735534668, 0.002085956744849682))

Equality of variance test (Levene): (LeveneResult(statistic=3.9344080210737533, pvalue=0.05756224804786322), LeveneResult(statistic=3.20614105163, pvalue=0.08458233272425814), LeveneResult(statistic=0.03467314551680419, pvalue=0.8536742012050543), LeveneResult(statistic=0.02865460566424988, pvalue=0.8668408598143552), LeveneResult(statistic=3.25658753013132, pvalue=0.08230339547366351))


In [59]:
#T-test for two independent samples to calculate between-groups differences in the three different states of connectivity. 

ttest_high_negative = stats.ttest_ind(td['prop_high_negative'], asd['prop_high_negative'])
ttest_moderate_negative = stats.ttest_ind(td['prop_moderate_negative'], asd['prop_moderate_negative'])
ttest_low_uncorrelated = stats.ttest_ind(td['prop_low_uncorrelated'], asd['prop_low_uncorrelated'])
ttest_moderate_positive = stats.ttest_ind(td['prop_moderate_positive'], asd['prop_moderate_positive'])
ttest_high_positive = stats.ttest_ind(td['prop_high_positive'], asd['prop_high_positive'])

ttest_dynStr = stats.ttest_ind(td['dynStr'],asd['dynStr'])
ttest_dynSD = stats.ttest_ind(td['dynSD'],asd['dynSD'])

print('ttest high negative:', ttest_high_negative)
print('ttest moderate negative:', ttest_moderate_negative)
print('ttest low uncorrelated:', ttest_low_uncorrelated)
print('ttest moderate negative:', ttest_moderate_positive)
print('ttest high positive:', ttest_high_positive)
print()
print('ttest dynFC strength:', ttest_dynStr)
print('ttest dynFC variability:', ttest_dynSD)

ttest high negative: Ttest_indResult(statistic=0.307092780269194, pvalue=0.7611288467217759)
ttest moderate negative: Ttest_indResult(statistic=-0.33329470051181187, pvalue=0.7414862979501019)
ttest low uncorrelated: Ttest_indResult(statistic=1.596939455873287, pvalue=0.1219185065796548)
ttest moderate negative: Ttest_indResult(statistic=1.2098655648976242, pvalue=0.23681708093463028)
ttest high positive: Ttest_indResult(statistic=-1.2095924144520758, pvalue=0.2369202121258409)

ttest dynFC strength: Ttest_indResult(statistic=-0.7379513858165466, pvalue=0.46690990376279784)
ttest dynFC variability: Ttest_indResult(statistic=0.20356955870497, pvalue=0.8402163912437461)


In [60]:
#Mann-Whitney non-parametric test for low and high count
print(stats.mannwhitneyu(td['prop_high_positive'], asd['prop_high_positive']))

MannwhitneyuResult(statistic=96.0, pvalue=0.35527871487876256)


In [63]:
#Standard deviation (SD) for each group to add error bars in the next plot. 
asd_prop_high_negative_std = np.std(asd['prop_high_negative'])
asd_prop_moderate_negative_std = np.std(asd['prop_moderate_negative'])
asd_prop_low_uncorrelated_std = np.std(asd['prop_low_uncorrelated'])
asd_prop_moderate_positive_std = np.std(asd['prop_moderate_positive'])
asd_prop_high_positive_std = np.std(asd['prop_high_positive'])

asd_std = np.array([asd_prop_high_negative_std, asd_prop_moderate_negative_std, asd_prop_low_uncorrelated_std, asd_prop_moderate_positive_std, asd_prop_high_positive_std])
print(asd_std)

td_prop_high_negative_std = np.std(td['prop_high_negative'])
td_prop_moderate_negative_std = np.std(td['prop_moderate_negative'])
td_prop_low_uncorrelated_std = np.std(td['prop_low_uncorrelated'])
td_prop_moderate_positive_std = np.std(td['prop_moderate_positive'])
td_prop_high_positive_std = np.std(td['prop_high_positive'])

td_std = np.array([td_prop_high_negative_std, td_prop_moderate_negative_std, td_prop_low_uncorrelated_std, td_prop_moderate_positive_std, td_prop_high_positive_std])
#asd_std = asd_std/2
print(td_std)

[0.18997931 0.07090061 0.07051449 0.04730578 0.21268368]
[0.0911803  0.03127975 0.07289784 0.0508711  0.10644448]


In [97]:
#Confidence intervals (CI) for each group to add error bars in the next plot. 

import statsmodels.stats.api as sms

asd_prop_high_negative_ci = sms.DescrStatsW(asd['prop_high_negative']).tconfint_mean()
asd_prop_moderate_negative_ci = sms.DescrStatsW(asd['prop_moderate_negative']).tconfint_mean()
asd_prop_low_uncorrelated_ci = sms.DescrStatsW(asd['prop_low_uncorrelated']).tconfint_mean()
asd_prop_moderate_positive_ci = sms.DescrStatsW(asd['prop_moderate_positive']).tconfint_mean()
asd_prop_high_positive_ci = sms.DescrStatsW(asd['prop_high_positive']).tconfint_mean()

asd_ci = np.array([asd_prop_high_negative_ci, asd_prop_moderate_negative_ci, asd_prop_low_uncorrelated_ci, asd_prop_moderate_positive_ci, asd_prop_high_positive_ci])
print(asd_ci)
print()
td_prop_high_negative_ci = sms.DescrStatsW(td['prop_high_negative']).tconfint_mean()
td_prop_moderate_negative_ci = sms.DescrStatsW(td['prop_moderate_negative']).tconfint_mean()
td_prop_low_uncorrelated_ci = sms.DescrStatsW(td['prop_low_uncorrelated']).tconfint_mean()
td_prop_moderate_positive_ci = sms.DescrStatsW(td['prop_moderate_positive']).tconfint_mean()
td_prop_high_positive_ci = sms.DescrStatsW(td['prop_high_positive']).tconfint_mean()

td_ci = np.array([td_prop_high_negative_ci, td_prop_moderate_negative_ci, td_prop_low_uncorrelated_ci, td_prop_moderate_positive_ci, td_prop_high_positive_ci])
print(td_ci)
td_ci[:,0]
td_ci[:,1]

[[0.1559389  0.38360192]
 [0.08411227 0.16907651]
 [0.18861658 0.27311811]
 [0.07879825 0.13548746]
 [0.13818952 0.39306048]]

[[0.23493616 0.3394686 ]
 [0.10171273 0.13757298]
 [0.23321362 0.31678638]
 [0.10089928 0.15921977]
 [0.12707931 0.24911117]]


array([0.3394686 , 0.13757298, 0.31678638, 0.15921977, 0.24911117])

In [123]:
asd_prop_high_negative_mean = np.mean(asd['prop_high_negative'])
asd_prop_moderate_negative_mean = np.mean(asd['prop_moderate_negative'])
asd_prop_low_uncorrelated_mean = np.mean(asd['prop_low_uncorrelated'])
asd_prop_moderate_positive_mean = np.mean(asd['prop_moderate_positive'])
asd_prop_high_positive_mean = np.mean(asd['prop_high_positive'])
      
td_prop_high_negative_mean = np.mean(td['prop_high_negative'])
td_prop_moderate_negative_mean = np.mean(td['prop_moderate_negative'])
td_prop_low_uncorrelated_mean = np.mean(td['prop_low_uncorrelated'])
td_prop_moderate_positive_mean = np.mean(td['prop_moderate_positive'])
td_prop_high_positive_mean = np.mean(td['prop_high_positive'])

states=['high negative', 'moderate negative', 'low uncorrelated', 'moderate positive', 'high positive']

fig = go.Figure(data=[
    go.Bar(name='ASD Group', x=states, y=[asd_prop_high_negative_mean, asd_prop_moderate_negative_mean, asd_prop_low_uncorrelated_mean, asd_prop_moderate_positive_mean, asd_prop_high_positive_mean]),
    go.Bar(name='TD Group', x=states, y=[td_prop_high_negative_mean, td_prop_moderate_negative_mean, td_prop_low_uncorrelated_mean, td_prop_moderate_positive_mean, td_prop_high_positive_mean]),
])

# Change the bar mode
fig.update_layout(barmode='group', title={
        'text': "Average proportion of windows in each state",
        'y':0.9,
        'x':0.5,
        'xanchor': 'center',
        'yanchor': 'top'},
    xaxis_title="States of connectivity",
    yaxis_title="Proportion of windows",)
fig.show()

In [108]:
go.Bar?

-----------------------------------------------
Next thing we want to test is if there are significant differences in the amount of time groups spend in a positive or negative state of FC.

-----------------------------------------------

-----------------------------------------
Next thing we want to obtain are the Mean Dwell Time (MDT) and the Probability of transition (PT). 

- MDT: average number of windows that were continuous on the time and attributed to the same state.
- PT: number of transitions from other states to a specific one over total available transitions(number of state changes).

--------------------------------------------

In [8]:
import itertools

MDT_high_negative = np.zeros(29)
MDT_moderate_negative = np.zeros(29)
MDT_low_uncorrelated = np.zeros(29)
MDT_moderate_positive = np.zeros(29)
MDT_high_positive = np.zeros(29)

PT_high_negative = np.zeros(29)
PT_moderate_negative = np.zeros(29)
PT_low_uncorrelated = np.zeros(29)
PT_moderate_positive = np.zeros(29)
PT_high_positive = np.zeros(29)

for i in np.arange(29):
    high_negative_condition = high_negative.loc[i]
    condition1 = high_negative_condition
    high_negative_consecutive_count = [sum( 1 for _ in group ) for key, group in itertools.groupby( condition1 ) if key]
    MDT1 = np.mean(high_negative_consecutive_count)
    MDT_high_negative[i] = MDT1
    size_high_negative = len(high_negative_consecutive_count) 
    
    moderate_negative_condition = moderate_negative.loc[i]
    condition2 = moderate_negative_condition
    moderate_negative_consecutive_count = [sum( 1 for _ in group ) for key, group in itertools.groupby( condition2 ) if key]
    MDT2 = np.mean(moderate_negative_consecutive_count)
    MDT_moderate_negative[i] = MDT2
    size_moderate_negative = len(moderate_negative_consecutive_count) 
    
    low_uncorrelated_condition = low_uncorrelated.loc[i]
    condition3 = low_uncorrelated_condition
    low_uncorrelated_consecutive_count = [sum( 1 for _ in group ) for key, group in itertools.groupby( condition3 ) if key]
    MDT3 = np.mean(low_uncorrelated_consecutive_count)
    MDT_low_uncorrelated[i] = MDT3
    size_low_uncorrelated = len(low_uncorrelated_consecutive_count) 
    
    moderate_positive_condition = moderate_positive.loc[i]
    condition4 = moderate_positive_condition
    moderate_positive_consecutive_count = [sum( 1 for _ in group ) for key, group in itertools.groupby( condition4 ) if key]
    MDT4 = np.mean(moderate_positive_consecutive_count)
    MDT_moderate_positive[i] = MDT4
    size_moderate_positive = len(moderate_positive_consecutive_count)
    
    high_positive_condition = high_positive.loc[i]
    condition5 = high_positive_condition
    high_positive_consecutive_count = [sum( 1 for _ in group ) for key, group in itertools.groupby( condition5 ) if key]
    MDT5 = np.mean(high_positive_consecutive_count)
    MDT_high_positive[i] = MDT5
    size_high_positive = len(high_positive_consecutive_count)
    
    
    total_changes = (size_high_negative + size_moderate_negative + size_low_uncorrelated + size_moderate_positive + size_high_positive) - 1
    
    PT_high_negative[i] = (size_high_negative / total_changes)
    PT_moderate_negative[i] = (size_moderate_negative / total_changes)
    PT_low_uncorrelated[i] = (size_low_uncorrelated / total_changes)
    PT_moderate_positive[i] = (size_moderate_positive / total_changes)
    PT_high_positive[i] = (size_high_positive / total_changes)
    
    
print(MDT_high_negative)
print(MDT_moderate_negative)
print(MDT_low_uncorrelated)
print(MDT_moderate_positive)
print(MDT_high_positive)
print()
print(PT_high_negative)
print(PT_moderate_negative)
print(PT_low_uncorrelated)
print(PT_moderate_positive)
print(PT_high_positive)

C:\Users\Laura\Anaconda3\lib\site-packages\numpy\core\fromnumeric.py:3118: RuntimeWarning:

Mean of empty slice.

C:\Users\Laura\Anaconda3\lib\site-packages\numpy\core\_methods.py:85: RuntimeWarning:

invalid value encountered in double_scalars



[13.85714286 13.          9.66666667 12.5         6.         11.25
  7.16666667         nan 10.66666667 16.125      17.71428571 10.25
 10.4        15.75       12.42857143 10.66666667 18.2         7.
 11.75       15.16666667  8.         15.8        10.66666667 14.
  8.33333333  9.6         8.6        15.         17.2       ]
[2.41666667 4.08333333 3.2        2.5        2.2        3.11111111
 4.5               nan 2.875      4.36363636 2.45454545 2.88888889
 2.91666667 1.92857143 2.61538462 2.66666667 2.11111111 2.53333333
 2.25       2.09090909 3.33333333 2.88888889 1.84615385 2.5
 2.75       3.22222222 1.5        3.28571429 2.23076923]
[ 5.77777778  4.4         5.375       6.          3.625       6.5
  5.28571429  8.          5.90909091  5.28571429  5.625       4.71428571
  4.          3.66666667 10.          7.5         5.85714286  5.75
  4.66666667  6.2         6.66666667  5.2         4.69230769  5.91666667
  4.72727273  3.75        4.63636364  5.33333333  8.6       ]
[2.71428571 1.8

In [9]:
#Creates new columns in the project data frame with the new indices that we have obtained for each state

project['MDT_high_negative'] = MDT_high_negative
project['MDT_moderate_negative'] = MDT_moderate_negative
project['MDT_low_uncorrelated'] = MDT_low_uncorrelated
project['MDT_moderate_positive'] = MDT_moderate_positive
project['MDT_high_positive'] = MDT_high_positive

project['PT_high_negative'] = PT_high_negative
project['PT_moderate_negative'] = PT_moderate_negative
project['PT_low_uncorrelated'] = PT_low_uncorrelated
project['PT_moderate_positive'] = PT_moderate_positive
project['PT_high_positive'] = PT_high_positive

project

,SRS_RAW_TOTAL,SRS_AWARENESS,SRS_COGNITION,SRS_COMMUNICATION,SRS_MOTIVATION,SRS_MANNERISMS,AQ_TOTAL,rest,Time1,Time2,...,MDT_high_negative,MDT_moderate_negative,MDT_low_uncorrelated,MDT_moderate_positive,MDT_high_positive,PT_high_negative,PT_moderate_negative,PT_low_uncorrelated,PT_moderate_positive,PT_high_positive
0,68,10,15,22,15,6,33,-0.372887,0.898793,0.928372,...,13.857143,2.416667,5.777778,2.714286,6.750000,0.184211,0.315789,0.236842,0.184211,0.105263
1,55,4,11,17,14,9,32,-0.252600,-0.794247,-0.678755,...,13.000000,4.083333,4.400000,1.875000,6.250000,0.175000,0.300000,0.250000,0.200000,0.100000
2,31,5,7,8,6,5,8,0.318661,-0.543222,-0.598560,...,9.666667,3.200000,5.375000,2.416667,15.285714,0.088235,0.147059,0.235294,0.352941,0.205882
3,72,9,15,26,10,12,30,0.344258,-0.016524,-0.025186,...,12.500000,2.500000,6.000000,3.230769,10.857143,0.052632,0.157895,0.289474,0.342105,0.184211
4,88,12,20,30,16,10,31,0.683666,0.743461,0.733636,...,6.000000,2.200000,3.625000,1.666667,31.400000,0.071429,0.178571,0.285714,0.321429,0.178571
5,77,10,10,24,18,15,36,-0.168494,-0.518876,-0.513324,...,11.250000,3.111111,6.500000,2.250000,7.666667,0.095238,0.214286,0.285714,0.285714,0.142857
6,119,16,24,36,20,23,40,-0.002792,-0.187023,-0.285734,...,7.166667,4.500000,5.285714,1.888889,9.000000,0.133333,0.311111,0.311111,0.200000,0.066667
7,68,8,16,19,13,12,23,0.666539,1.057666,0.894216,...,NaN,NaN,8.000000,3.250000,19.125000,0.000000,0.000000,0.173913,0.521739,0.347826
8,98,13,19,29,21,16,37,-0.102764,0.818552,0.972918,...,10.666667,2.875000,5.909091,2.400000,16.000000,0.083333,0.222222,0.305556,0.277778,0.138889
9,80,9,14,28,15,14,33,-0.601657,-0.668932,-0.732067,...,16.125000,4.363636,5.285714,3.333333,NaN,0.285714,0.392857,0.250000,0.107143,0.000000


In [10]:
#Redefine td and asd so they have the new columns added to the 'project' data frame. 
asd = project.loc[:13]
td = project.loc[14:] 
asd.replace(np.nan, 0, inplace = True)
td.replace(np.nan, 0, inplace = True)

#T-test between-groups to test for significant differences in the indices for each state.
ttest_MDT_high_negative = stats.ttest_ind(td['MDT_high_negative'],asd['MDT_high_negative'])
ttest_MDT_moderate_negative = stats.ttest_ind(td['MDT_moderate_negative'],asd['MDT_moderate_negative'])
ttest_MDT_low_uncorrelated = stats.ttest_ind(td['MDT_low_uncorrelated'],asd['MDT_low_uncorrelated'])
ttest_MDT_moderate_positive = stats.ttest_ind(td['MDT_moderate_positive'],asd['MDT_moderate_positive'])
ttest_MDT_high_positive = stats.ttest_ind(td['MDT_high_positive'],asd['MDT_high_positive'])

ttest_PT_high_negative = stats.ttest_ind(td['PT_high_negative'],asd['PT_high_negative'])
ttest_PT_moderate_negative = stats.ttest_ind(td['PT_moderate_negative'],asd['PT_moderate_negative'])
ttest_PT_low_uncorrelated = stats.ttest_ind(td['PT_low_uncorrelated'],asd['PT_low_uncorrelated'])
ttest_PT_moderate_positive = stats.ttest_ind(td['PT_moderate_positive'],asd['PT_moderate_positive'])
ttest_PT_high_positive = stats.ttest_ind(td['PT_high_positive'],asd['PT_high_positive'])

print('ttest MDT high_negative connectivity:', ttest_MDT_high_negative)
print('ttest MDT moderate_negative connectivity:', ttest_MDT_moderate_negative)
print('ttest MDT low_uncorrelated connectivity:', ttest_MDT_low_uncorrelated)
print('ttest MDT moderate_positive connectivity:', ttest_MDT_moderate_positive)
print('ttest MDT high_positive connectivity:', ttest_MDT_high_positive)
print()   
print('ttest PT high_negative connectivity:', ttest_PT_high_negative)
print('ttest PT moderate_negative connectivity:', ttest_PT_moderate_negative)
print('ttest PT low_uncorrelated connectivity:', ttest_PT_low_uncorrelated)
print('ttest PT moderate_positive connectivity:', ttest_PT_moderate_positive)
print('ttest PT high_positive connectivity:', ttest_PT_high_positive)

ttest MDT high_negative connectivity: Ttest_indResult(statistic=0.7488225950723808, pvalue=0.46043836054598497)
ttest MDT moderate_negative connectivity: Ttest_indResult(statistic=-0.9113606384102115, pvalue=0.37017084386171817)
ttest MDT low_uncorrelated connectivity: Ttest_indResult(statistic=1.2397687271079711, pvalue=0.22572857725273357)
ttest MDT moderate_positive connectivity: Ttest_indResult(statistic=2.6554001772909914, pvalue=0.013126007248723095)
ttest MDT high_positive connectivity: Ttest_indResult(statistic=-0.2908892940447804, pvalue=0.773358835161563)

ttest PT high_negative connectivity: Ttest_indResult(statistic=0.7315766301444665, pvalue=0.47072941214553277)
ttest PT moderate_negative connectivity: Ttest_indResult(statistic=1.4095306166160901, pvalue=0.17009329619162672)
ttest PT low_uncorrelated connectivity: Ttest_indResult(statistic=0.8848386315064402, pvalue=0.38405598906052096)
ttest PT moderate_positive connectivity: Ttest_indResult(statistic=-1.2740183250279022,

C:\Users\Laura\Anaconda3\lib\site-packages\pandas\core\frame.py:4263: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [ ]:
#Standard deviation (SD) for each group to add error bars in the next plot. 
asd_MDT_high_negative_std = np.std(asd['MDT_high_negative'])
asd_MDT_moderate_negative_std = np.std(asd['MDT_moderate_negative'])
asd_MDT_low_uncorrelated_std = np.std(asd['MDT_low_uncorrelated'])
asd_MDT_moderate_positive_std = np.std(asd['MDT_moderate_positive'])
asd_MDT_high_positive_std = np.std(asd['MDT_high_positive'])

asd_std_MDT = np.array([asd_MDT_high_negative_std, asd_MDT_moderate_negative_std, asd_MDT_low_uncorrelated_std, asd_MDT_moderate_positive_std, asd_MDT_high_positive_std])
print(asd_std_MDT)

td_prop_high_negative_std = np.std(td['MDT_high_negative'])
td_prop_moderate_negative_std = np.std(td['MDT_moderate_negative'])
td_prop_low_uncorrelated_std = np.std(td['MDT_low_uncorrelated'])
td_prop_moderate_positive_std = np.std(td['MDT_moderate_positive'])
td_prop_high_positive_std = np.std(td['MDT_high_positive'])

td_std_MDT = np.array([td_MDT_high_negative_std, td_MDT_moderate_negative_std, td_MDT_low_uncorrelated_std, td_MDT_moderate_positive_std, td_MDT_high_positive_std])
#asd_std = asd_std/2
print(td_std_MDT)

In [11]:
asd_MDT_high_negative_mean = np.mean(asd['MDT_high_negative'])
asd_MDT_moderate_negative_mean = np.mean(asd['MDT_moderate_negative'])
asd_MDT_low_uncorrelated_mean = np.mean(asd['MDT_low_uncorrelated'])
asd_MDT_moderate_positive_mean = np.mean(asd['MDT_moderate_positive'])
asd_MDT_high_positive_mean = np.mean(asd['MDT_high_positive'])
      
td_MDT_high_negative_mean = np.mean(td['MDT_high_negative'])
td_MDT_moderate_negative_mean = np.mean(td['MDT_moderate_negative'])
td_MDT_low_uncorrelated_mean = np.mean(td['MDT_low_uncorrelated'])
td_MDT_moderate_positive_mean = np.mean(td['MDT_moderate_positive'])
td_MDT_high_positive_mean = np.mean(td['MDT_high_positive'])



import plotly.graph_objects as go
states=['high negative', 'moderate negative', 'low uncorrelated', 'moderate positive', 'high positive']

fig = go.Figure(data=[
    go.Bar(name='ASD Group', x=states, y=[asd_MDT_high_negative_mean, asd_MDT_moderate_negative_mean, asd_MDT_low_uncorrelated_mean, asd_MDT_moderate_positive_mean, asd_MDT_high_positive_mean]),
    go.Bar(name='TD Group', x=states, y=[td_MDT_high_negative_mean, td_MDT_moderate_negative_mean, td_MDT_low_uncorrelated_mean, td_MDT_moderate_positive_mean, td_MDT_high_positive_mean]),
])
# Change the bar mode
fig.update_layout(barmode='group', title={
        'text': "Mean Dwell Time",
        'y':0.9,
        'x':0.5,
        'xanchor': 'center',
        'yanchor': 'top'},
    xaxis_title="States of connectivity",
    yaxis_title="MDT",)
fig.show()

In [12]:
asd_PT_high_negative_mean = np.mean(asd['PT_high_negative'])
asd_PT_moderate_negative_mean = np.mean(asd['PT_moderate_negative'])
asd_PT_low_uncorrelated_mean = np.mean(asd['PT_low_uncorrelated'])
asd_PT_moderate_positive_mean = np.mean(asd['PT_moderate_positive'])
asd_PT_high_positive_mean = np.mean(asd['PT_high_positive'])
      
td_PT_high_negative_mean = np.mean(td['PT_high_negative'])
td_PT_moderate_negative_mean = np.mean(td['PT_moderate_negative'])
td_PT_low_uncorrelated_mean = np.mean(td['PT_low_uncorrelated'])
td_PT_moderate_positive_mean = np.mean(td['PT_moderate_positive'])
td_PT_high_positive_mean = np.mean(td['PT_high_positive'])

states=['high negative', 'moderate negative', 'low uncorrelated', 'moderate positive', 'high positive']

fig = go.Figure(data=[
    go.Bar(name='ASD Group', x=states, y=[asd_PT_high_negative_mean, asd_PT_moderate_negative_mean, asd_PT_low_uncorrelated_mean, asd_PT_moderate_positive_mean, asd_PT_high_positive_mean]),
    go.Bar(name='TD Group', x=states, y=[td_PT_high_negative_mean, td_PT_moderate_negative_mean, td_PT_low_uncorrelated_mean, td_PT_moderate_positive_mean, td_PT_high_positive_mean]),
])
# Change the bar mode
fig.update_layout(barmode='group', title={
        'text': "Probability of Transition",
        'y':0.9,
        'x':0.5,
        'xanchor': 'center',
        'yanchor': 'top'},
    xaxis_title="States of connectivity",
    yaxis_title="PT",)
fig.show()

In [13]:
print(stats.pearsonr(asd['dynSD'], asd['SRS_RAW_TOTAL']))
print(stats.pearsonr(asd['dynSD'], asd['SRS_AWARENESS']))
print(stats.pearsonr(asd['dynSD'], asd['SRS_COGNITION']))
print(stats.pearsonr(asd['dynSD'], asd['SRS_COMMUNICATION']))
print(stats.pearsonr(asd['dynSD'], asd['SRS_MOTIVATION']))
print(stats.pearsonr(asd['dynSD'], asd['SRS_MANNERISMS'])) #*****
print(stats.pearsonr(asd['dynSD'], asd['AQ_TOTAL']))

(-0.31402336301134937, 0.27422731860047844)
(-0.2068977431892865, 0.4778889868048628)
(-0.2047761461528389, 0.48251436178743506)
(-0.2469440488791229, 0.39469629755797747)
(-0.08903039236124002, 0.7621457139019011)
(-0.5394115288334178, 0.046513636952887905)
(-0.27950075405108965, 0.33315997671358555)


In [14]:
print(stats.pearsonr(project['dynStr'], project['SRS_RAW_TOTAL']))
print(stats.pearsonr(project['dynStr'], project['SRS_AWARENESS']))
print(stats.pearsonr(project['dynStr'], project['SRS_COGNITION']))
print(stats.pearsonr(project['dynStr'], project['SRS_COMMUNICATION']))
print(stats.pearsonr(project['dynStr'], project['SRS_MOTIVATION']))
print(stats.pearsonr(project['dynStr'], project['SRS_MANNERISMS']))
print(stats.pearsonr(project['dynStr'], project['AQ_TOTAL']))

(0.14045485846177014, 0.46739984045836286)
(0.268005438095101, 0.15983204454521174)
(0.23721229374110697, 0.21534035210290806)
(0.0937272882780886, 0.6286684843170446)
(0.09749007174272532, 0.6148883365991507)
(0.0023589053515449127, 0.9903104844783333)
(0.007193004196037016, 0.9704594680851525)


In [15]:
print(stats.pearsonr(asd['prop_high_negative'], asd['SRS_RAW_TOTAL']))
print(stats.pearsonr(asd['prop_high_negative'], asd['SRS_AWARENESS']))
print(stats.pearsonr(asd['prop_high_negative'], asd['SRS_COGNITION']))
print(stats.pearsonr(asd['prop_high_negative'], asd['SRS_COMMUNICATION']))
print(stats.pearsonr(asd['prop_high_negative'], asd['SRS_MOTIVATION']))
print(stats.pearsonr(asd['prop_high_negative'], asd['SRS_MANNERISMS']))
print(stats.pearsonr(asd['prop_high_negative'], asd['AQ_TOTAL']))

#asd['prop_high_negative'] = prop_high_negative
#asd['prop_moderate_negative'] = prop_moderate_negative
#asd['prop_low_uncorrelated'] = prop_low_uncorrelated
#asd['prop_moderate_positive'] = prop_moderate_positive
#asd['prop_high_positive'] = prop_high_positive

(0.0955881875364073, 0.7451373947167228)
(-0.2236476249067037, 0.44212589907306016)
(-0.2104091411244259, 0.4702802179722738)
(0.20868782460746016, 0.4740028168550911)
(0.3116911331820583, 0.2779994503339413)
(0.1700483062178752, 0.5611048063498298)
(0.2876698277365957, 0.3186185082147295)


In [16]:
print(stats.pearsonr(asd['prop_moderate_negative'], asd['SRS_RAW_TOTAL']))
print(stats.pearsonr(asd['prop_moderate_negative'], asd['SRS_AWARENESS']))
print(stats.pearsonr(asd['prop_moderate_negative'], asd['SRS_COGNITION']))
print(stats.pearsonr(asd['prop_moderate_negative'], asd['SRS_COMMUNICATION']))
print(stats.pearsonr(asd['prop_moderate_negative'], asd['SRS_MOTIVATION']))
print(stats.pearsonr(asd['prop_moderate_negative'], asd['SRS_MANNERISMS']))
print(stats.pearsonr(asd['prop_moderate_negative'], asd['AQ_TOTAL'])) 

(0.3741476563710348, 0.1875373951995065)
(0.19671931571187257, 0.5002696459760837)
(0.21320540161909196, 0.464262942939226)
(0.3623341225908058, 0.20297041588956818)
(0.3010871726256017, 0.29553420679831544)
(0.44007169028427273, 0.1153265855711299)
(0.5175548033929134, 0.05802480676978931)


In [17]:
print(stats.pearsonr(asd['prop_low_uncorrelated'], asd['SRS_RAW_TOTAL']))
print(stats.pearsonr(asd['prop_low_uncorrelated'], asd['SRS_AWARENESS']))
print(stats.pearsonr(asd['prop_low_uncorrelated'], asd['SRS_COGNITION']))
print(stats.pearsonr(asd['prop_low_uncorrelated'], asd['SRS_COMMUNICATION']))
print(stats.pearsonr(asd['prop_low_uncorrelated'], asd['SRS_MOTIVATION']))
print(stats.pearsonr(asd['prop_low_uncorrelated'], asd['SRS_MANNERISMS']))
print(stats.pearsonr(asd['prop_low_uncorrelated'], asd['AQ_TOTAL']))

(0.27811987998667254, 0.33565414299298213)
(0.4230708203231386, 0.13174997621693396)
(0.06203716578879501, 0.8331356960889661)
(0.22610539830519438, 0.43699314485952)
(0.1648657112102149, 0.5732743564915127)
(0.35170240208475073, 0.21752575824909387)
(0.31621694028501174, 0.27070729807991867)


In [18]:
print(stats.pearsonr(asd['prop_moderate_positive'], asd['SRS_RAW_TOTAL']))
print(stats.pearsonr(asd['prop_moderate_positive'], asd['SRS_AWARENESS']))
print(stats.pearsonr(asd['prop_moderate_positive'], asd['SRS_COGNITION']))
print(stats.pearsonr(asd['prop_moderate_positive'], asd['SRS_COMMUNICATION']))
print(stats.pearsonr(asd['prop_moderate_positive'], asd['SRS_MOTIVATION'])) #*****
print(stats.pearsonr(asd['prop_moderate_positive'], asd['SRS_MANNERISMS']))
print(stats.pearsonr(asd['prop_moderate_positive'], asd['AQ_TOTAL']))

(-0.33485648088293873, 0.2418885430247249)
(-0.16383474893946287, 0.5757081041346935)
(-0.11795475911724417, 0.6879747548475245)
(-0.33403474141975553, 0.24311781947253905)
(-0.5576094130241553, 0.038281120610779554)
(-0.19525886211921079, 0.5035199692531994)
(-0.39759158134948186, 0.1591918350320998)


In [19]:
print(stats.pearsonr(asd['prop_high_positive'], asd['SRS_RAW_TOTAL']))
print(stats.pearsonr(asd['prop_high_positive'], asd['SRS_AWARENESS']))
print(stats.pearsonr(asd['prop_high_positive'], asd['SRS_COGNITION']))
print(stats.pearsonr(asd['prop_high_positive'], asd['SRS_COMMUNICATION']))
print(stats.pearsonr(asd['prop_high_positive'], asd['SRS_MOTIVATION']))
print(stats.pearsonr(asd['prop_high_positive'], asd['SRS_MANNERISMS']))
print(stats.pearsonr(asd['prop_high_positive'], asd['AQ_TOTAL']))

(-0.22784029049019364, 0.43338807286298625)
(0.030367206886053433, 0.917921226828059)
(0.12254073021369216, 0.6764324242371441)
(-0.30786572117569555, 0.28425265804165645)
(-0.3094238621096948, 0.28169575046632195)
(-0.3717739758044564, 0.1905760405024712)
(-0.4459005351421526, 0.11003331701645176)


In [20]:
print(stats.pearsonr(asd['MDT_moderate_positive'], asd['SRS_RAW_TOTAL']))
print(stats.pearsonr(asd['MDT_moderate_positive'], asd['SRS_AWARENESS']))
print(stats.pearsonr(asd['MDT_moderate_positive'], asd['SRS_COGNITION']))
print(stats.pearsonr(asd['MDT_moderate_positive'], asd['SRS_COMMUNICATION']))
print(stats.pearsonr(asd['MDT_moderate_positive'], asd['SRS_MOTIVATION']))
print(stats.pearsonr(asd['MDT_moderate_positive'], asd['SRS_MANNERISMS']))
print(stats.pearsonr(asd['MDT_moderate_positive'], asd['AQ_TOTAL']))

(-0.24144280403999263, 0.40564705628790615)
(-0.24027851579289322, 0.40798468870123655)
(-0.07422693699248725, 0.8008983392744512)
(-0.1579123463095593, 0.5897701238728746)
(-0.4224692578483572, 0.13235838005637304)
(-0.15327652229755268, 0.6008720838913933)
(-0.14722797691261222, 0.6154788564015311)


In [29]:
stats.pearsonr(project['MDT_moderate_positive'], project['dynSD'])

(-0.15645429839953112, 0.41766395211452756)

In [33]:
print(stats.pearsonr(td['prop_high_negative'], td['SRS_RAW_TOTAL'])) #****
print(stats.pearsonr(td['prop_high_negative'], td['SRS_AWARENESS'])) #****
print(stats.pearsonr(td['prop_high_negative'], td['SRS_COGNITION']))
print(stats.pearsonr(td['prop_high_negative'], td['SRS_COMMUNICATION']))
print(stats.pearsonr(td['prop_high_negative'], td['SRS_MOTIVATION'])) #****
print(stats.pearsonr(td['prop_high_negative'], td['SRS_MANNERISMS']))
print(stats.pearsonr(td['prop_high_negative'], td['AQ_TOTAL']))

(-0.5254526885166505, 0.044267443273376075)
(-0.5267085465171615, 0.043671011998730115)
(-0.2565780613249218, 0.3559553921704392)
(-0.4895359262645884, 0.06400376536892362)
(-0.6394971387304886, 0.010255320065036412)
(-0.3369325692163623, 0.2194349325756509)
(-0.38576808038698535, 0.15556820648908848)


In [21]:
#project.to_csv('processed_States_Kaiser_rvaTPJ_no_rvpTPJ_374_001_008.csv')